# Limpieza de datos

Para los datos del Estado de México que son historicos se tiene que realizar lo siguiente:


*  Eliminar columnas: Precipitación, Evaporación
*  Si la temperatura máxima y mínima en algún registro es nulo, eliminarlo
*  Si es nulo alguno de los dos valores de temperatura, calcular el promedio con las dos temperaturas anteriores y las dos temperaturas posteriores



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import chardet

import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import os

In [ ]:
# Autenticación y creación del cliente PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Descarga los archivos y los convierte en dataframes

# ID de la carpeta compartida que deseas listar
carpeta_id = '1pvA46f64RB_3CpOXZQ68iMGgWBZkkqs-'

# Lista para almacenar los nombres de los archivos
files_dataframes = []

# Obtener la lista de archivos en la carpeta compartida
file_list = drive.ListFile({'q': "'{}' in parents".format(carpeta_id)}).GetList()

for archivo in file_list:
    try:
        # Descargar el archivo y guardarlo en el directorio actual
        archivo.GetContentFile(archivo['title'])
        # Leer el archivo descargado y crear un DataFrame
        df = pd.read_excel(archivo['title'])
        # Agregar el DataFrame al array
        files_dataframes.append(df)
        print("Archivo descargado y DataFrame creado:", archivo['title'])
    except Exception as e:
        print("Error al descargar y crear DataFrame para el archivo:", archivo['title'])
        print(e)

Archivo descargado y DataFrame creado: PueblaSan Nicolas (1).xlsx
Archivo descargado y DataFrame creado: Manuel_utf8.xlsx
Archivo descargado y DataFrame creado: Atlacomulco_utf8.xlsx
Archivo descargado y DataFrame creado: Ixtapaluca_utf8.xlsx
Archivo descargado y DataFrame creado: Oro_utf8.xlsx
Archivo descargado y DataFrame creado: Toluca_utf8.xlsx
Archivo descargado y DataFrame creado: Valle_utf8.xlsx
Archivo descargado y DataFrame creado: Edo Amecameca.xlsx
Archivo descargado y DataFrame creado: PueblaSan Nicolas.xlsx
Archivo descargado y DataFrame creado: laguna.xlsx
Archivo descargado y DataFrame creado: cerro.xlsx
Archivo descargado y DataFrame creado: nevado.xlsx


In [ ]:
files_dataframes[0].dtypes

Fecha                datetime64[ns]
Precipitacion               float64
Evaporacion                  object
TemperaturaMaxima           float64
TemperaturaMinima           float64
dtype: object

In [ ]:
def calcular_valor(fila):
  if fila['Precipitacion'] == 0.0:
    return 0
  elif fila['Precipitacion'] > 0.0 and fila['Precipitacion'] < 15.0:
    return 1
  elif (fila['Precipitacion'] > 15.0 and fila['Precipitacion'] < 30.0):
    return 2
  else:
    return 3

In [ ]:
def limpiar_datos(df):
    columnas = ['Precipitacion', 'TemperaturaMaxima', 'TemperaturaMinima']
    df[columnas] = df[columnas].apply(pd.to_numeric, errors="coerce")
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors="coerce")
    df = df.drop('Evaporacion', axis = 1)
    df = df.dropna(subset=['TemperaturaMaxima','TemperaturaMinima'], how = "all")

    df[columnas] = df[columnas].interpolate(method='linear', limit_direction='both')
    df['Categoria'] = df.apply(calcular_valor, axis = 1)

    return df

In [ ]:
# Crea un ThreadPoolExecutor con 4 hilos
with ThreadPoolExecutor(max_workers=4) as executor:
    # Ejecuta la función de limpieza para cada archivo
    resultados = list(executor.map(limpiar_datos, files_dataframes))

df_final = pd.concat(resultados, ignore_index = True)

In [ ]:
df_final.to_csv('resultados_modulos.csv', index=False)
# Guarda el DataFrame en un archivo CSV dentro de la carpeta compartida
#final_df.to_csv(os.path.join(shared_folder_path, 'resultados_modulos.csv'), index=False)

print(df_final.head(50))

        Fecha  Precipitacion  TemperaturaMaxima  TemperaturaMinima  \
0  2007-01-05           22.0               12.0               10.0   
1  2007-02-05           32.0               14.0                8.0   
2  2007-03-05           14.0               12.0               10.0   
3  2007-04-05            0.0               14.0                8.0   
4  2007-05-05            0.0               14.0               10.0   
5  2007-06-05            0.0               12.0               10.0   
6  2007-07-05            0.0               12.0                8.0   
7  2007-08-05            0.0               14.0                8.0   
8  2007-09-05            0.0               16.0                8.0   
9  2007-10-05            0.0               12.0               10.0   
10 2007-11-05            0.0               14.0               10.0   
11 2007-12-05            0.0               12.0               10.0   
12 2007-05-13            0.0               12.0                8.0   
13 2007-05-14       

In [ ]:
print(df_final.isna().sum())

Fecha                0
Precipitacion        0
TemperaturaMaxima    0
TemperaturaMinima    0
dtype: int64
